#### LEAD AND LAG

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Windows_Functions_Lead_Lag").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/22 22:02:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/02/22 22:02:26 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


- Create sample dataframe

In [2]:
simpleData = (
    ("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("James", "Sales", 3000),
    ("Saif", "Sales", 4100),
    ("Maria", "Finance", 3000),
    ("scott", "Finance", 3000),
    ("Jen", "Marketing", 3000),
    ("Kumar", "Marketing", 2000),
)
columns = ["employee_name", "department", "salary"]
df = spark.createDataFrame(data=simpleData, schema=columns)
df.show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        James|     Sales|  3000|
|         Saif|     Sales|  4100|
|        Maria|   Finance|  3000|
|        scott|   Finance|  3000|
|          Jen| Marketing|  3000|
|        Kumar| Marketing|  2000|
+-------------+----------+------+



#### Window Defination

In [3]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

In [5]:
windowSpec = Window.partitionBy("department").orderBy("salary")

#### LAG Window Function

In [6]:
from pyspark.sql.functions import lag

In [7]:
df.withColumn("lag", lag("salary", 1).over(window=windowSpec)).show()

+-------------+----------+------+----+
|employee_name|department|salary| lag|
+-------------+----------+------+----+
|        Maria|   Finance|  3000|NULL|
|        scott|   Finance|  3000|3000|
|        Kumar| Marketing|  2000|NULL|
|          Jen| Marketing|  3000|2000|
|        James|     Sales|  3000|NULL|
|        James|     Sales|  3000|3000|
|       Robert|     Sales|  4100|3000|
|         Saif|     Sales|  4100|4100|
|      Michael|     Sales|  4600|4100|
+-------------+----------+------+----+



#### LEAD window Functions

In [8]:
from pyspark.sql.functions import lead

In [10]:
df.withColumn("lead", lead("salary", 1).over(window=windowSpec)).show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|        Maria|   Finance|  3000|3000|
|        scott|   Finance|  3000|NULL|
|        Kumar| Marketing|  2000|3000|
|          Jen| Marketing|  3000|NULL|
|        James|     Sales|  3000|3000|
|        James|     Sales|  3000|4100|
|       Robert|     Sales|  4100|4100|
|         Saif|     Sales|  4100|4600|
|      Michael|     Sales|  4600|NULL|
+-------------+----------+------+----+

